In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib
import pandas as pd
import re
import datetime
import numpy as np
import pickle
import requests
from time import sleep
import pandas as pd
import random
import os

In [2]:
version = "1.0.1"
datadir = "data/"
archivedir = "data/Archive"

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import ui
from selenium.webdriver.support.ui import WebDriverWait, Select

from selenium.webdriver.firefox.options import Options
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

In [4]:
def gedjournalarticles(filename):
    # open old file
    olddf = pd.DataFrame({'url': [] })
    try:
        olddf = pd.read_pickle(os.path.join(archivedir, filename))
        print(olddf.tail())
    except:
        pass

    driver.get("https://www.jstage.jst.go.jp/browse/sola/list/-char/en")
    
    # get all of the issue
    def issues():
        issues = []
        # get all of the Article on this page
        content = driver.page_source
        # load the page content in BeautifulSoup
        soup = BeautifulSoup(content, features="lxml")
        # get all of the issue div
        for a in soup.find_all('a', {'class': 'bluelink-style'}):
            # if it is this journal
            if re.search("Volume", " ".join([ str(x) for x in a.contents ]) ):
                issues.append(a['href'])        
        
        return  issues    
    allissue = issues()
    allissue = list(set(allissue))
    
    def articles():
        articles = []
        nexpage = False
        # get all of the Article on this page
        content = driver.page_source
        # load the page content in BeautifulSoup
        soup = BeautifulSoup(content, features="lxml")
        # get all of the issue div
        for a in soup.find_all('a'):
            # if it is this journal
            if re.search("^https://doi.org", " ".join([ str(x) for x in a.contents ]) ):
                articles.append(a['href'])
        # check there is new issue
        for a in driver.find_elements_by_name('a'):
            if '>' == a.getText():
                nexpage = True
                # set to nexpage
                a.click()
                sleep(3)        
        return  articles, nexpage
    
    # iterate over isses and get article links
    articellinks = []
    for issue in allissue:
        driver.get(issue)
        sleep(2)
        print(driver.current_url, end='\r') 
        # get the issue article
        nextpage = True
        allissue = []
        while nextpage:
            thispagearticles, nextpage = articles()
            articellinks = articellinks + thispagearticles
            print(driver.current_url, end='\r')        
    articellinks = list(set(articellinks))
    
    # filter out old articles
    oldarticles = olddf['url']
    articellinks = list( set(articellinks).difference(set(oldarticles) ))
    print(len(articellinks), "new article found!")
    
    # iterate over article 
    urls = [ None for _ in range(len(articellinks))]
    titles = [ None for _ in range(len(articellinks))]
    abstracts = [ None for _ in range(len(articellinks))]
    writers = [ None for _ in range(len(articellinks))]
    dates = [ None for _ in range(len(articellinks))]
    dois = [  None for _ in range(len(articellinks)) ]
    keywords = [  None for _ in range(len(articellinks)) ]
    for idx in range(len(articellinks)):
        # print percentiage of the process
        print( str(np.round(100*idx/len(articellinks),2))+"%" , end='\r')
        url =  articellinks[idx]
        # load page
        driver.get(articellinks[idx])
        sleep(3)        
        urls[idx] = driver.current_url
        # get the issue article
        # get all of the Article on this page
        content = driver.page_source
        # load the page content in BeautifulSoup
        soup = BeautifulSoup(content, features="lxml")
        # process metadata
        keyword = []
        writter = []
        notpaper = False
        for m in soup.find_all("meta"):
            if m.has_attr("name"):
                if m['name'] == 'title':
                    titles[idx] = m['content']
                if m['name'] == "citation_keywords":
                    keyword.append( m['content'] )
                if m['name'] == "citation_online_date":
                    dates[idx] = datetime.datetime.strptime(m['content'], "%Y/%m/%d").strftime('%Y-%m-%d')
                if m['name'] == 'citation_author':
                    writter.append(m['content'])
                if re.search('citation_doi', m['name']):
                    dois[idx] = m['content']                    
        if len(writter) > 0:
            writers[idx] =  "#".join(writter)
        if len(keyword) > 0:
            keywords[idx] =  "#".join(keyword)
            
        # get abstract
        abstract = ""
        if notpaper:
            continue
        else:
            for div in soup.find_all(id='article-overiew-abstract-wrap'):
                for c in div.find_all(['p', 'script']):
                    if c.name == "p":                    
                        for tag in  c.contents:
                            if re.search("<span", str(tag)):
                                continue
                            else:
                                match = re.match("<script.+>", str(tag))
                                if match is not None:
                                    abstract = abstract + " $"+str(tag).split('>')[1].split('<')[0] + "$ "
                                else:
                                    abstract = abstract + str(tag).replace("\n", '').replace("\t", '')
                # if still not abstract it is possible it is not in p
                if len(abstract) == 0:
                    for tag in div.contents:
                        if re.search("<span", str(tag)) or re.search("<h3", str(tag)) :
                            continue
                        else:
                            match = re.match("<script.+>", str(tag))
                            if match is not None:
                                abstract = abstract + " $"+str(tag).split('>')[1].split('<')[0] + "$ "
                            else:
                                abstract = abstract + str(tag).replace("\n", '').replace("\t", '')
            abstracts[idx] = abstract       
        
    
    # make df
    df = pd.DataFrame({'url': urls, 
                       'journal_title': "SOLA", 
                       'journal_eissn': "1349-6476",
                       'journal_pissn': '',
                       'category': "Meteorology. Climatology",                                              
                      })
    # extend the df
    df['title'] = titles
    df['doi'] = dois
    df['abstract'] = abstracts
    df['writer'] = writers
    df['publishdate'] = dates
    df['keyword'] = keywords
    
    # drop where is no abstract
    df = df[ df['abstract'].notna()]
    # drop where title is Editorial
    df = df[df['title'] != 'Editorial']
    # drop where title "The SOLA Award in *""
    df = df[df.title.str.contains('^(?!The SOLA Award in)', regex= True, na=False)]
    
    
    # check there was a not Arhived but previously loaded file
    adf = None
    try:
        adf = pd.read_pickle(os.path.join(datadir, filename))
    except:
        pass
    if adf is not None:
        df = pd.concat([df, adf])
        df.drop_duplicates(inplace=True)
    
    return df, olddf

In [ ]:
filename = 'journal_SOLA_'+version+'.pandas'
    
# search for articles
df, olddf = gedjournalarticles( filename )

# save data
df.to_pickle(os.path.join(datadir, filename))

print(df.head())


762 new article found!.go.jp/browse/sola/6/0/_contents/-char/enen


In [ ]:
print(df.iloc[0]['abstract'])

In [ ]:
print(df.iloc[0])

In [ ]:
driver.close()